In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import datetime

In [2]:
filename = "stateTaxes.xlsx"
state_taxes = pd.read_excel(filename, parse_dates = True)
state_taxes = state_taxes[state_taxes["State"] != 0.0]
tax_names = state_taxes.iloc[0]
state_taxes = state_taxes.iloc[1:]
state_taxes["Year"] = pd.to_datetime(state_taxes["Year"].astype(int), format = "%Y")
state_taxes["Name"] = state_taxes["Name"].str[:2]
tax_names = tax_names.iloc[4:]
state_taxes.rename(columns = {taxid:taxname for taxid, taxname in tax_names.items()}, inplace = True)
state_taxes = state_taxes.replace("X","0").replace("T0", "TX")
for key in tax_names:
    state_taxes[key] = state_taxes[key].astype(int)
state_taxes[state_taxes[tax_names]<0] = 0

state_taxes.set_index(["Name", "Year"], inplace = True)
state_taxes

,,State,FY Ending Date,Total Taxes,Property Tax (T01),Tot Sales & Gr Rec Tax,Total Gen Sales Tax (T09),Total Select Sales Tax,Alcoholic Beverage Tax (T10),Amusement Tax (T11),Insurance Premium Tax (T12),...,Occup and Bus Lic NEC (T28),Other License Taxes (T29),Total Income Taxes,Individual Income Tax (T40),Corp Net Income Tax (T41),Total Other Taxes,Death and Gift Tax (T50),Docum and Stock Tr Tax (T51),Severance Tax (T53),Taxes NEC (T99)
Name,Year,,,,,,,,,,,,,,,,,,,,,
AL,2023-01-01,1.0,93023.0,16713126,553812,7952183,4607610,3344573,282537,0,563244,...,70795,1,7470605,5910871,1559734,111783,0,57424,54359,0
AK,2023-01-01,2.0,63023.0,3271369,128961,292255,0,292255,41621,21369,63446,...,16252,10092,444240,0,444240,2296473,0,0,2296473,0
AZ,2023-01-01,3.0,63023.0,23363426,1304903,14862923,12509337,2353586,91425,8152,811228,...,182588,2314,6323049,4779485,1543564,190828,0,0,18526,172302
AR,2023-01-01,4.0,63023.0,12758766,1424090,6668614,4980130,1688484,78615,92034,374949,...,135129,1120,3985931,3147770,838161,245911,0,73715,98360,73836
CA,2023-01-01,5.0,63023.0,220591306,3400425,75400445,53565932,21834513,421647,0,3690456,...,4013036,4364,126315948,96379294,29936654,2209603,348,0,135200,2074055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VA,1942-01-01,47.0,63042.0,65682,4884,35745,0,35745,6552,0,2133,...,0,3831,9394,3121,6273,2096,725,0,25,1346
WA,1942-01-01,48.0,33142.0,90388,4618,73516,40248,33268,2964,0,1831,...,0,1393,0,0,0,3277,3138,0,0,139
WV,1942-01-01,49.0,63042.0,56851,196,44763,30961,13802,988,0,1039,...,0,990,2474,2474,0,1196,516,0,0,680


In [3]:
years = [1985,
 1990,
 1995,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

fips_dict = {
 '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR',
 '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE',
 '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID',
 '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS',
 '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
 '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
 '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV',
 '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY',
 '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK',
 '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC',
 '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
 '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
 '55': 'WI', '56': 'WY'}
# '60': 'AS', '66': 'GU',
#  '69': 'MP', '72': 'PR', '78': 'VI'}
state_df = pd.DataFrame(state_dict, index = ["Abbr"]).T
fips_df = pd.DataFrame(fips_dict, index = ["State"]).T
state_names, state_abbrev = state_df.index, list(state_df["Abbr"].values)
state_df.T# state_abbrev

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Abbr,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY


In [4]:
import time
def gather_data(data_codes, 
                start, 
                end = datetime.datetime.today(), 
                freq = "M"):
    i = 0
    # dct.items() calls key and value that key points to
    for key, val in data_codes.items():
        time.sleep(.51)
        if i == 0:
            # Create dataframe for first variable, then rename column
            df = web.DataReader(
                val, "fred", start, end).resample(freq).mean()
            df.rename(columns = {val:key}, inplace = True) 
            # setting i to None will cause the next block of code to execute,
            # placing data within df instead of creating a new dataframe for
            # each variable
            i = None
        else:
            # If dataframe already exists, add new column
            df[key] = web.DataReader(val, "fred", start, end).resample(freq).mean()

    return df

In [ ]:
pop_codes  = {state: f"{state}POP" for state in state_dict.values()}
med_home_price_codes = {state: f"MEDLISPRI{state}" for state in state_dict.values()}
index_home_price_codes = {state: f"{state}STHPI" for state in state_dict.values()}
codes = {"Population":pop_codes,
         # "Median Home Price":med_home_price_codes,
         "Home Price Index":index_home_price_codes}
panels = {}

# Select start and end dates
start = datetime.datetime(2002, 1, 1)
end = datetime.datetime.today()
# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days
for name, code in codes.items():
    code_data = gather_data(data_codes = codes[name], start = start, end = end, freq = "A")
    panels[name] = code_data.reset_index()
    panels[name]["DATE"] = pd.to_datetime(panels[name]["DATE"].astype(str).str[:4].astype(int), format = "%Y")
    panels[name] = panels[name].melt(id_vars=["DATE"], 
            var_name="State", 
            value_name="Population").set_index(["State", "DATE"]).round(2)
panels

In [ ]:
storage_options = {'User-Agent': 'Mozilla/5.0'}
url = "https://www.fraserinstitute.org/sites/default/files/economic-freedom-of-north-america-2022-for-website-allgovs.xlsx"
efw = pd.read_excel(url, storage_options=storage_options, sheet_name=None)
efw = efw["Overall-scores(allgovs)"].dropna(subset = "Unnamed: 104").rename(columns = {"Overall allg Scores":"State"})


In [ ]:
panel = efw[efw["Unnamed: 104"].str.contains(", US")]
panel = panel[["State"] + years]
panel["State"] = panel["State"].apply(lambda x:state_dict[x])
panel = panel.melt(id_vars=["State"], 
        var_name="Year", 
        value_name="EFW")
panel["Year"] = pd.to_datetime(panel["Year"],format = "%Y")
panel = panel.set_index(["State", "Year"]).round(2)
panel

In [ ]:
GDP = pd.read_csv("StateGDP.csv")#.rename(columns = {year:datetime.datetime(int(year),1,1) for year in GDP.keys()[8:]})
GDP_years = list(GDP.keys())[-24:]
GDP = GDP[GDP["GeoName"].isin(state_names)]
GDP["State"] = GDP["GeoName"].apply(lambda x: state_dict[x])
GDP = GDP[["State","LineCode"] + GDP_years]
GDP = GDP.melt(id_vars=["State", "LineCode"], 
        var_name="Year", 
        value_name="GDP")
GDP["Year"] = pd.to_datetime(GDP["Year"], format = "%Y")

GDP.set_index(["State", "Year"], inplace = True)
panel.loc[panel.index.isin(GDP.index),"GDP"] = GDP["GDP"][GDP["LineCode"]==1].astype(float)
for name in panels.keys():
    panel[name] = panels[name]

In [ ]:
tax_panel = state_taxes.copy()
for name, val in panel.items():
    tax_panel[name] = val
tax_panel["GDP Per Capita"] = tax_panel["GDP"].div(tax_panel["Population"])
tax_panel.dropna().head()

In [ ]:
growth_vars = ["GDP", "Population", "GDP Per Capita", "EFW"]
# nontax_names = [name for name in pct_state_taxes.keys() if name not in tax_names.values]
pct_state_taxes = tax_panel.copy()
pct_state_taxes[tax_names] = pct_state_taxes[tax_names].apply(lambda x: x.div(pct_state_taxes["Total Taxes"]))
pct_state_taxes[growth_vars] = np.log(tax_panel[growth_vars]).groupby("Name").diff()
pct_state_taxes.dropna().head()

In [ ]:
pct_state_taxes_5year = pct_state_taxes.copy()
pct_state_taxes_5year[growth_vars] = np.log(tax_panel[growth_vars]).groupby("Name").diff(5)
pct_state_taxes_5year.dropna()


In [ ]:
dfs = {"Levels":tax_panel, 
       "Rates":pct_state_taxes,
       "5YearRates": pct_state_taxes_5year,
       "DiffRates":pct_state_taxes.groupby("Name").diff().dropna()[growth_vars + list(tax_names.values)],
       "5YearDiffRates":pct_state_taxes_5year.groupby("Name").diff(5).dropna()[growth_vars + list(tax_names.values)],
      }
dfs

In [ ]:
for name, df in dfs.items():
    for income_tax in ("Individual Income Tax (T40)", "Corp Net Income Tax (T41)", "Total Income Taxes", "GDP", "Population", "GDP Per Capita", "EFW"):
        fig, ax = plt.subplots(figsize = (15,15))
        x = "Property Tax (T01)"
        df.plot.scatter(x = x, y = income_tax, ax = ax)
        corrstr = df[[x, income_tax]].corr().round(2).loc[x, income_tax].astype(str)
        ax.set_title(f"{name} Corr: {corrstr}")

In [ ]:
import pingouin
def plot_scatter_corr(plot_df, title, corr = "corr"):
    corr_df = getattr(plot_df, corr)().round(2)
    num_keys = len(plot_df.keys())
    fig, axs = plt.subplots(num_keys, num_keys, figsize = (20,20))
    for i, key1 in enumerate(plot_df.keys()):
        for j, key2 in enumerate(plot_df.keys()):
            
            if i < j:
                ax = axs[j][i]
                plot_df.plot.scatter(x = key1, y =key2, ax = ax)
            elif i > j:
                ax = axs[j][i]
                ax.text(.5,.5, corr_df.astype(str).loc[key2, key1],
                       ha = "center", va = "center",
                       fontsize = 28)
                ax.set_xticklabels([])
            else:
                ax = axs[i][j]
                plot_df[[key1]].hist(density = True, ax = ax)
                if i == 0: ax.set_ylabel(key1)
                # if i == num_keys - 1: ax.set_xlabel(key1)
            
            if i >0:
                ax.set_ylabel("")
                ax.set_yticklabels([])
            else:
                ax.set_ylabel(key2.replace(" ", "\n"), fontsize = 24)
            if j < num_keys - 1: 
                ax.set_xticklabels([])
            if j > 0:
                ax.set_title("")
            else:
                ax.set_title(key1.replace(" ", "\n"), fontsize = 24)
            ax.set_xlabel("")
    plt.suptitle(name, fontsize = 40, y = .965)
    return fig, ax
plot_keys = ["Total Income Taxes", "GDP", "Population", "GDP Per Capita", "EFW",
             "Property Tax (T01)", "Individual Income Tax (T40)", 
             "Corp Net Income Tax (T41)"]
for name, df in dfs.items():
    plot_df = df[plot_keys]
    fig, ax = plot_scatter_corr(plot_df, name, corr = "corr")
    # use limited set for pcorr
    # plot_scatter_corr(plot_df, name, corr = "pcorr")
    